In [2]:
pip install opencv-python

  Using cached opencv_python-4.5.5.64-cp36-abi3-win_amd64.whl (35.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install gluoncv

  Using cached gluoncv-0.10.5-py2.py3-none-any.whl (1.3 MB)
  Using cached portalocker-2.4.0-py2.py3-none-any.whl (16 kB)
  Using cached autocfg-0.0.8-py3-none-any.whl (13 kB)
  Using cached yacs-0.1.8-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: python-dateutil>=2.1 in c:\users\wjd72\anaconda3\envs\py36\lib\site-packages (from matplotlib->gluoncv) (2.8.2)



In [1]:
from __future__ import division
import argparse, time, logging, os, math, tqdm, cv2

import numpy as np
import mxnet as mx
from mxnet import gluon, nd, image
from mxnet.gluon.data.vision import transforms

import matplotlib.pyplot as plt

import gluoncv as gcv
from gluoncv import data
from gluoncv.data import mscoco
from gluoncv.model_zoo import get_model
from gluoncv.data.transforms.pose import detector_to_alpha_pose,heatmap_to_coord_alpha_pose, detector_to_simple_pose, heatmap_to_coord
from gluoncv.utils.viz import cv_plot_image, cv_plot_keypoints

In [2]:
ctx = mx.cpu()
# detector_name = "ssd_512_mobilenet1.0_coco"
detector_name= "yolo3_mobilenet1.0_coco"
detector = get_model(detector_name, pretrained=True, ctx=ctx)

In [3]:
detector.reset_class(classes=['person'], reuse_weights={'person':'person'})
detector.hybridize()

In [4]:
# estimators = get_model('simple_pose_resnet18_v1b', pretrained='ccd24037', ctx=ctx)
estimators = get_model('alpha_pose_resnet101_v1b_coco', pretrained=True, ctx=ctx)
estimators.hybridize()

In [5]:
# cap = cv2.VideoCapture(0)
# time.sleep(1)  ### letting the camera autofocus

In [6]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

True

# Realtime-AlphaPose 적용

In [ ]:
import cv2

capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)


# used to record the time when we processed last frame
prev_frame_time = 0
 
# used to record the time at which we processed current frame
new_frame_time = 0


while cv2.waitKey(33) < 0:
    ret, frame = capture.read()
    frame = mx.nd.array(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).astype('uint8')
#     x, frame = data.transforms.presets.yolo.load_test(frame, short=512)
#     x, frame = gcv.data.transforms.presets.ssd.transform_test(frame, short=512, max_size=350)
    x, frame = gcv.data.transforms.presets.yolo.transform_test(frame, short=512, max_size=350)
    x = x.as_in_context(ctx)
    class_IDs, scores, bounding_boxs = detector(x)

    pose_input, upscale_bbox = detector_to_alpha_pose(frame, class_IDs, scores, bounding_boxs,
                                                       output_shape=(128, 96), ctx=ctx)
    
    
#     if len(upscale_bbox) > 0:
#         predicted_heatmap = estimators(pose_input)
#         pred_coords, confidence = heatmap_to_coord_alpha_pose(predicted_heatmap, upscale_bbox)

#         img = cv_plot_keypoints(frame, pred_coords, confidence, class_IDs, bounding_boxs, scores,
#                                 box_thresh=0.5, keypoint_thresh=0.2)
        
#         print(pred_coords)


    # font which we will be using to display FPS
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    # time when we finish processing for this frame
    new_frame_time = time.time()


    # Calculating the fps
 
    # fps will be number of frame processed in given time frame
    # since their will be most of time error of 0.001 second
    # we will be subtracting it to get more accurate result
    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time
 
    # converting the fps into integer
    fps = int(fps)
 
    # converting the fps to string so that we can display it on frame
    # by using putText function
    fps = str(fps)
 
    # putting the FPS count on the frame
    cv2.putText(frame, fps, (7, 70), font, 3, (100, 255, 0), 3, cv2.LINE_AA)
 
    # press 'Q' if you want to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    cv2.imshow("VideoFrame", frame)

capture.release()
cv2.destroyAllWindows()

In [8]:
detector_to_alpha_pose(frame, class_IDs, scores, bounding_boxs,
                                                       output_shape=(128, 96), ctx=ctx)

(
 [[[[-0.406 -0.406 -0.406 ... -0.406 -0.406 -0.406]
    [-0.406 -0.406 -0.406 ... -0.406 -0.406 -0.406]
    [-0.406 -0.406 -0.406 ... -0.406 -0.406 -0.406]
    ...
    [-0.406 -0.406 -0.406 ... -0.406 -0.406 -0.406]
    [-0.406 -0.406 -0.406 ... -0.406 -0.406 -0.406]
    [-0.406 -0.406 -0.406 ... -0.406 -0.406 -0.406]]
 
   [[-0.457 -0.457 -0.457 ... -0.457 -0.457 -0.457]
    [-0.457 -0.457 -0.457 ... -0.457 -0.457 -0.457]
    [-0.457 -0.457 -0.457 ... -0.457 -0.457 -0.457]
    ...
    [-0.457 -0.457 -0.457 ... -0.457 -0.457 -0.457]
    [-0.457 -0.457 -0.457 ... -0.457 -0.457 -0.457]
    [-0.457 -0.457 -0.457 ... -0.457 -0.457 -0.457]]
 
   [[-0.48  -0.48  -0.48  ... -0.48  -0.48  -0.48 ]
    [-0.48  -0.48  -0.48  ... -0.48  -0.48  -0.48 ]
    [-0.48  -0.48  -0.48  ... -0.48  -0.48  -0.48 ]
    ...
    [-0.48  -0.48  -0.48  ... -0.48  -0.48  -0.48 ]
    [-0.48  -0.48  -0.48  ... -0.48  -0.48  -0.48 ]
    [-0.48  -0.48  -0.48  ... -0.48  -0.48  -0.48 ]]]]
 <NDArray 1x3x128x96 @cpu(0)>